In [ ]:
from IPython.display import Image
import json
import folium
import openrouteservice as ors
from openrouteservice.isochrones import isochrones

client = ors.Client(key='5b3ce3597851110001cf6248fadb94ffd01d455fb2bb86e68caf8f87')

lat = 47.502394 #ZHAW
lon = 8.7263853 #ZHAW

# ORS-Isochrones with statistics

isochrone_geojson = isochrones(client, (lon, lat), intervals=[600, 1200, 1800], 
attributes={'total_pop'}, profile='driving-car')

# Extract population data
pop_sting = 'Population for: '
for x in range(0, len(isochrone_geojson['features'])):
    total_pop = (isochrone_geojson['features'][x]['properties']['total_pop'])
    pop_sting += 'Isochrone {} is {} | '.format(x, total_pop)

# Folium map with marker
map_osm = folium.Map(location=[lat, lon]) # Folium map
folium.GeoJson(isochrone_geojson).add_to(map_osm) # Add data to map
folium.Marker([lat, lon], popup=pop_sting).add_to(map_osm) # Add marker to map 

In [ ]:
#Imports
from openrouteservice.directions import directions
import pandas as pd

# Prepare data
df_ladestationen = pd.read_csv('../Ausgabedateien/locations_Ladestationen.csv')
df = pd.read_csv('../Quelldateien/Test_Routing.csv')
coords = []
for x in range(0, len(df_ladestationen.index -1)):
    coords.append(df_ladestationen.loc[x, 'Latitude':'Longitude'])
print(df_ladestationen)

map_osm1 = folium.Map(location=[47.494075, 8.715564], zoom_start=14) # Folium Map

In [ ]:
#Process data
time = 0 # Total time counter
distance = 0 # Total distance counter
for x in range(0, len(coords)-1):
    coordinates = ((coords[x][1], coords[x][0]), (8.726374, 47.50241)) # Coordinates
    geojson = directions(client, coordinates, format_out='geojson') # ORS
    for y in range(0, len(geojson['features'][0]['properties']['segments'])): # Features
        time += geojson['features'][0]['properties']['segments'][y]['duration']
        distance += geojson['features'][0]['properties']['segments'][y]['distance']
    folium.GeoJson(geojson).add_to(map_osm1) # Add to map
    print('OK! {} | {}'.format(x, coordinates))

In [ ]:
#Process data
time = 0 # Total time counter
distance = 0 # Total distance counter
for x in range(0, len(coords)-1):
    coordinates = ((coords[x][1], coords[x][0]), (coords[x+1][1], coords[x+1][0])) # Coordinates
    geojson = directions(client, coordinates, format_out='geojson') # ORS
    for y in range(0, len(geojson['features'][0]['properties']['segments'])): # Features
        time += geojson['features'][0]['properties']['segments'][y]['duration']
        distance += geojson['features'][0]['properties']['segments'][y]['distance']
    folium.GeoJson(geojson).add_to(map_osm1) # Add to map
    print('OK! {} | {}'.format(x, coordinates))

In [ ]:
print('Total Length: {} km'.format(round(distance/1000, 0)))
print('Total Time: {} Min'.format(round(time/60)))